# KNN - Analysis

#### Import Dataset and Packages

In [40]:
# Import Packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [41]:
# Import the Data

df_movies = pd.read_csv('/workspaces/gustavolima-kkneighbours/data/raw/tmdb_5000_movies.csv')
df_credits = pd.read_csv('/workspaces/gustavolima-kkneighbours/data/raw/tmdb_5000_credits.csv')

df_movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [42]:
df_credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [43]:
# Check Dataset Movies Shape 
df_movies.shape


(4803, 20)

In [44]:
# Check Dataset Credits Shape
df_credits.shape

(4803, 4)

#### Create a SQL Database

In [45]:
# Import Packages
import sqlite3

In [46]:
# Create DB
conn = sqlite3.connect('/workspaces/gustavolima-kkneighbours/data/raw/Movies.sqlite')


# Create and Convert the DataFrames to SQL
df_movies.to_sql('movies', conn, if_exists='replace', index=False)
df_credits.to_sql('credits', conn, if_exists='replace', index=False)

# Query to create new table
query = '''
    SELECT *
    FROM movies
    INNER JOIN credits
    USING (title);
'''
# Create new Database
tmdb = pd.read_sql_query(query, conn)
conn.close()

##### Clean the new DataFrame

In [47]:
tmdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [48]:
# Remove unwanted Columns from DataFrame
tmdb = tmdb[['movie_id','title','overview','genres','keywords','cast','crew']]

In [49]:
tmdb.shape

(4809, 7)

In [50]:
# Check if there are Null Data
tmdb.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

#### Treat the Information of the Dataset

In [51]:
# Import JSON package
import json

In [52]:
tmdb["genres"] = tmdb["genres"].apply(lambda x: [item["name"] for item in json.loads(x)] if pd.notna(x) else None)
tmdb["keywords"] = tmdb["keywords"].apply(lambda x: [item["name"] for item in json.loads(x)] if pd.notna(x) else None)
tmdb["cast"] = tmdb["cast"].apply(lambda x: [item["name"] for item in json.loads(x)][:3] if pd.notna(x) else None)
tmdb["crew"] = tmdb["crew"].apply(lambda x: next((item["name"] for item in json.loads(x) if item["job"] == "Director"), None) if pd.notna(x) else None)
tmdb["overview"] = tmdb["overview"].apply(lambda x: [x] if pd.notna(x) else None)

tmdb

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In the 22nd century, a paraplegic Marine is d...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron
1,285,Pirates of the Caribbean: At World's End,"[Captain Barbossa, long believed to be dead, h...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski
2,206647,Spectre,[A cryptic message from Bond’s past sends him ...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes
3,49026,The Dark Knight Rises,[Following the death of District Attorney Harv...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",Christopher Nolan
4,49529,John Carter,"[John Carter is a war-weary, former military c...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",Andrew Stanton
...,...,...,...,...,...,...,...
4804,9367,El Mariachi,[El Mariachi just wants to play his guitar and...,"[Action, Crime, Thriller]","[united states–mexico barrier, legs, arms, pap...","[Carlos Gallardo, Jaime de Hoyos, Peter Marqua...",Robert Rodriguez
4805,72766,Newlyweds,[A newlywed couple's honeymoon is upended by t...,"[Comedy, Romance]",[],"[Edward Burns, Kerry Bishé, Marsha Dietlein]",Edward Burns
4806,231617,"Signed, Sealed, Delivered","[""Signed, Sealed, Delivered"" introduces a dedi...","[Comedy, Drama, Romance, TV Movie]","[date, love at first sight, narration, investi...","[Eric Mabius, Kristin Booth, Crystal Lowe]",Scott Smith
4807,126186,Shanghai Calling,[When ambitious New York attorney Sam is sent ...,[],[],"[Daniel Henney, Eliza Coupe, Bill Paxton]",Daniel Hsia


In [53]:
# Remove Spaces
def remove_spaces_from_list(lst):
    return [item.replace(" ", "") for item in lst] if isinstance(lst, list) else lst

tmdb[["genres", "cast", "keywords"]] = tmdb[["genres", "cast", "keywords"]].applymap(remove_spaces_from_list)
tmdb["crew"] = tmdb["crew"].str.replace(" ", "")

tmdb.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In the 22nd century, a paraplegic Marine is d...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",JamesCameron
1,285,Pirates of the Caribbean: At World's End,"[Captain Barbossa, long believed to be dead, h...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",GoreVerbinski
2,206647,Spectre,[A cryptic message from Bond’s past sends him ...,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",SamMendes
3,49026,The Dark Knight Rises,[Following the death of District Attorney Harv...,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",ChristopherNolan
4,49529,John Carter,"[John Carter is a war-weary, former military c...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",AndrewStanton


In [54]:
# Convert the Lists to Strings in the Dataset
def list_to_string(lst):
    return ",".join(lst) if isinstance(lst, list) else ""

tmdb["genres"] = tmdb["genres"].apply(list_to_string)
tmdb["keywords"] = tmdb["keywords"].apply(list_to_string)
tmdb["cast"] = tmdb["cast"].apply(list_to_string)
tmdb["overview"] = tmdb["overview"].apply(list_to_string)

In [55]:
# Create a Single Line content in a "Tags" Colum
tmdb['overview'] = tmdb['overview'].str.replace(",", "")
tmdb["tags"] = tmdb["overview"] + " " + tmdb["genres"] + " " + tmdb["keywords"]  + " " + tmdb["cast"] + " " + tmdb["crew"]
tmdb["tags"] = tmdb["tags"].str.replace(",", " ")

In [56]:
tmdb.iloc[0].tags

'In the 22nd century a paraplegic Marine is dispatched to the moon Pandora on a unique mission but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

#### Use KNN to Recommend New Movies

In [59]:
from sklearn.feature_extraction.text import CountVectorizer

tmdb["tags"] = tmdb["tags"].fillna("")
cv = CountVectorizer(max_features=5000 ,stop_words='english')
vectors = cv.fit_transform(tmdb['tags']).toarray()

In [60]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(vectors)

In [64]:
def recommend(movie):
    movie_lower = movie.lower()  # Convert the input to lowercase
    movie_index = tmdb[tmdb["title"].str.lower() == movie_lower].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    
    for i in movie_list:
        print(tmdb.iloc[i[0]].title)

In [65]:
user_input = input("Enter a movie name: ")
recommend(user_input)

Aliens
Meet Dave
Alien³
Escape from Planet Earth
Star Trek: Insurrection
